# **IMPORTANT**: Now due to the lack of ressources on the free tier of colab. You can only use one domaine. In this example I have the laptop domain working for the moment. To change to the restaurants uncomment there block of codes that load the aspect & the sentiment models. Then comment out the laptops one.

# **Thank you and Enjoy**.

#Fouad AURAG

# 0- Donwload the necessary libraries

In [ ]:
# If you are in Kaggle
# %%capture
# !mamba install --force-reinstall aiohttp -y
# !pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
# !pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# # Temporary fix for https://github.com/huggingface/datasets/issues/6753
# !pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

# import os
# os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# If you are in Colab
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
!pip install pyngrok
!pip install flask flask_cors

# 1- Import the necessary Libraries

In [ ]:
from pyngrok import ngrok
from flask import Flask, request, jsonify, session
from flask_cors import CORS
import ast

In [ ]:
NGROK_AUTH_TOKEN = "NGROK_AUTH_TOKEN_HERE"
PORT_NUMBER = 5200

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(PORT_NUMBER).public_url

In [ ]:
print("Add the following api url to Aspectify, so you can accss the backend")
print("Backend API URL: ",public_url)

#### on va utiliser flask comme un framework de backend

In [ ]:
app = Flask(__name__)
CORS(app)

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

restaurants_aspect_model, restaurants_aspect_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "auragFouad/mistral7b_aspect_restaurants",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(restaurants_aspect_model)

==((====))==  Unsloth 2024.9: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(

In [ ]:
# laptops_aspect_model, laptops_aspect_tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "auragFouad/mistral7b_aspect_laptops", # YOUR MODEL YOU USED FOR TRAINING
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )
# FastLanguageModel.for_inference(laptops_aspect_model) # Enable native 2x faster inference

In [ ]:
restaurants_sentiment_model, restaurants_sentiment_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "auragFouad/llama3_sentiment_restaurants", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(restaurants_sentiment_model) # Enable native 2x faster inference

==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [ ]:
# laptops_sentiment_model, laptops_sentiment_tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "auragFouad/llama3_sentiment_laptops", # YOUR MODEL YOU USED FOR TRAINING
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )
# FastLanguageModel.for_inference(laptops_sentiment_model)

# Now we define the alpaca prompt and instructions for aspect prediction and

---

sentiment classification on Restaurants and Laptops domains

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
restaurants_aspect_instruction_prompt = "Recognize all aspect terms in the given review delimited by triple quotes. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the product/service. choose from the following aspects [RESTAURANT#GENERAL,SERVICE#GENERAL,FOOD#QUALITY,FOOD#STYLE_OPTIONS,DRINKS#STYLE_OPTIONS,DRINKS#PRICES,RESTAURANT#PRICES,RESTAURANT#MISCELLANEOUS,AMBIENCE#GENERAL,FOOD#PRICES,LOCATION#GENERAL,DRINKS#QUALITY]. Answer in the format [aspect] without any explanation, in case of multiple aspects append them into the list of aspects as a list like this [aspect_1, aspect_2]. If no aspect term exists, then only answer []."

In [ ]:
laptops_aspect_instruction_prompt = "Recognize all aspect terms in the given review delimited by triple quotes. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the product/service. choose from the following aspects [LAPTOP#GENERAL, BATTERY#OPERATION_PERFORMANCE, CPU#OPERATION_PERFORMANCE, GRAPHICS#GENERAL, HARD_DISC#DESIGN_FEATURES, LAPTOP#OPERATION_PERFORMANCE, LAPTOP#USABILITY, LAPTOP#PORTABILITY, LAPTOP#PRICE, LAPTOP#QUALITY, LAPTOP#DESIGN_FEATURES, LAPTOP#MISCELLANEOUS, OS#DESIGN_FEATURES, BATTERY#QUALITY, SUPPORT#QUALITY, COMPANY#GENERAL, DISPLAY#USABILITY, DISPLAY#DESIGN_FEATURES, OS#GENERAL, LAPTOP#CONNECTIVITY, DISPLAY#QUALITY, OS#USABILITY, MOUSE#DESIGN_FEATURES, SOFTWARE#MISCELLANEOUS, KEYBOARD#DESIGN_FEATURES, KEYBOARD#GENERAL, SOFTWARE#GENERAL, OPTICAL_DRIVES#QUALITY, GRAPHICS#QUALITY, HARD_DISC#QUALITY, WARRANTY#GENERAL, MOUSE#QUALITY, MULTIMEDIA_DEVICES#USABILITY, MULTIMEDIA_DEVICES#QUALITY, PORTS#QUALITY, DISPLAY#GENERAL, POWER_SUPPLY#QUALITY, POWER_SUPPLY#OPERATION_PERFORMANCE, HARDWARE#QUALITY, DISPLAY#OPERATION_PERFORMANCE, MULTIMEDIA_DEVICES#GENERAL, MULTIMEDIA_DEVICES#DESIGN_FEATURES, KEYBOARD#USABILITY, KEYBOARD#QUALITY, POWER_SUPPLY#DESIGN_FEATURES, SHIPPING#QUALITY, MOUSE#OPERATION_PERFORMANCE, MOUSE#USABILITY, OS#MISCELLANEOUS, SHIPPING#PRICE, KEYBOARD#OPERATION_PERFORMANCE, SUPPORT#MISCELLANEOUS, CPU#QUALITY, GRAPHICS#DESIGN_FEATURES, OS#OPERATION_PERFORMANCE, MEMORY#DESIGN_FEATURES, SOFTWARE#QUALITY, SOFTWARE#USABILITY, SOFTWARE#DESIGN_FEATURES, OS#QUALITY, SOFTWARE#OPERATION_PERFORMANCE, PORTS#DESIGN_FEATURES, CPU#MISCELLANEOUS, MOTHERBOARD#QUALITY, SOFTWARE#PRICE, MOUSE#GENERAL, MULTIMEDIA_DEVICES#OPERATION_PERFORMANCE, SUPPORT#PRICE, WARRANTY#PRICE, FANS_COOLING#DESIGN_FEATURES, MULTIMEDIA_DEVICES#MISCELLANEOUS, FANS_COOLING#QUALITY, POWER_SUPPLY#MISCELLANEOUS, PORTS#OPERATION_PERFORMANCE, GRAPHICS#MISCELLANEOUS, FANS_COOLING#OPERATION_PERFORMANCE, BATTERY#MISCELLANEOUS, HARDWARE#GENERAL, OPTICAL_DRIVES#OPERATION_PERFORMANCE, HARDWARE#OPERATION_PERFORMANCE, CPU#DESIGN_FEATURES]. Answer in the format [aspect] without any explanation, in case of multiple aspects append them into the list of aspects as a list like this [aspect_1, aspect_2]. If no aspect term exists, then only answer []."

In [ ]:
restaurants_sentiment_instruction_prompt = "Given a sentence delimited by triple quotes between \"\" with a specific aspect next to it in between \"\", predict the sentiment polarity (positive, negative, or neutral) associated with that aspect in the given sentence. Answer back with the sentiment without any explanation.", # instruction

In [ ]:
laptops_sentiment_instruction_prompt = "Given a sentence delimited by triple quotes between \"\" with a specific aspect next to it in between \"\", predict the sentiment polarity (positive, negative, or neutral) associated with that aspect in the given sentence. Answer back with the sentiment without any explanation.", # instruction

## Now  i have 4 models:

    - aspect_restaurants_mistral_7b
    - aspect_laptop_mistral_7b
    - sentiment_restaurants_llama3_8b
    - sentiment_laptop_llama3_8b
    
For each task/domain we define a function for inference, and another for preprocessing the result

# Aspect/Restaurants

In [ ]:
# Inference with restaurants_aspect_model/restaurants_aspect_tokenizer/restaurants_aspect_instruction_prompt
def get_aspect_restaurants(user_description):
    input_text = user_description
    inputs = restaurants_aspect_tokenizer(
    [
      alpaca_prompt.format(
          restaurants_aspect_instruction_prompt,
          input_text, # input
          "", # output - leave this blank for generation!
      )
    ], return_tensors = "pt").to("cuda")

    outputs = restaurants_aspect_model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    output = restaurants_aspect_tokenizer.batch_decode(outputs)
    return output

# Preprocessing the model output
def get_aspect_restaurants_list(output):

    prediction = output
    prediction = prediction[0]
    prediction = prediction.split("### Response:\n")
    prediction = prediction[1]
    prediction = prediction.split("<|end_of_text|>")
    prediction = prediction[0]
    sides = prediction.split("]")
    prediction = sides[0]
    if prediction.endswith("'"):
        prediction = ast.literal_eval(prediction + "]")
    else:
        prediction = prediction + "']"
        prediction = ast.literal_eval(prediction)

    return prediction

# Aspect/Laptops

In [ ]:
# Inference with laptops_aspect_model/laptops_aspect_tokenizer/laptops_aspect_instruction_prompt
def get_aspect_laptops(user_description):
    input_text = user_description
    inputs = laptops_aspect_tokenizer(
    [
      alpaca_prompt.format(
          laptops_aspect_instruction_prompt,
          input_text, # input
          "", # output - leave this blank for generation!
      )
    ], return_tensors = "pt").to("cuda")

    outputs = laptops_aspect_model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    output = laptops_aspect_tokenizer.batch_decode(outputs)

    return output

# Preprocessing the model output
def get_aspect_laptops_list(output):

    prediction = output
    # prediction = ast.literal_eval(prediction)
    prediction = prediction[0]
    prediction = prediction.split("### Response:\n")
    prediction = prediction[1]
    prediction = prediction.split("<|end_of_text|>")
    prediction = prediction[0]
    sides = prediction.split("]")
    prediction = sides[0]
    if prediction.endswith("'"):
        prediction = ast.literal_eval(prediction + "]")
    else:
        prediction = prediction + "']"
        prediction = ast.literal_eval(prediction)

    return prediction

# Sentiment/Restaurants

In [ ]:
# Get users prediction.
def get_aspect_sentiment_restaurants(output, aspect):
    aspect_sentiment_list = []
    predictions_list = []

    prediction = output
    prediction = prediction[0]
    prediction = prediction.split("### Response:\n")
    prediction = prediction[1]
    prediction = prediction.split("<|end_of_text|>")
    prediction = prediction[0]

    if "," in prediction:
        predictions = prediction.split(",")
        for prediction in predictions:
            predictions_list.append(prediction)
    else:
        predictions_list.append(prediction)

    for i in range(len(predictions_list)):
        aspect_sentiment_prediction = (aspect[i], predictions_list[i])
        aspect_sentiment_list.append(aspect_sentiment_prediction)

    return aspect_sentiment_list


# Inference with restaurants_sentiment_model/restaurants_sentiment_tokenizer/restaurants_sentiment_instruction_prompt
def get_sentiment_restaurants(user_description, restaurants_aspects_list):
    sentence = user_description
    aspect = restaurants_aspects_list

    input_text = f"The sentence is: \"{sentence}\" and the Aspect is: \"{aspect}\""

    inputs = restaurants_sentiment_tokenizer(
    [
      alpaca_prompt.format(
          restaurants_sentiment_instruction_prompt,
          input_text, # input
          "", # output - leave this blank for generation!
      )
    ], return_tensors = "pt").to("cuda")

    outputs = restaurants_sentiment_model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    output = restaurants_sentiment_tokenizer.batch_decode(outputs)

    aspect_sentiment_restaurants = get_aspect_sentiment_restaurants(output, restaurants_aspects_list)

    return aspect_sentiment_restaurants

# Sentiment/Laptop

In [ ]:
# Get users prediction.
def get_aspect_sentiment_laptops(output, aspect):
    aspect_sentiment_list = []
    predictions_list = []

    prediction = output
    prediction = prediction[0]
    prediction = prediction.split("### Response:\n")
    prediction = prediction[1]
    prediction = prediction.split("<|end_of_text|>")
    prediction = prediction[0]

    if "," in prediction:
        predictions = prediction.split(",")
        for prediction in predictions:
            predictions_list.append(prediction)
    else:
        predictions_list.append(prediction)

    for i in range(len(predictions_list)):
        aspect_sentiment_prediction = (aspect[i], predictions_list[i])
        aspect_sentiment_list.append(aspect_sentiment_prediction)

    return aspect_sentiment_list



# Inference with laptops_sentiment_model/laptops_sentiment_tokenizer/laptops_sentiment_instruction_prompt
def get_sentiment_laptops(user_description, laptops_aspects_list):

    sentence = user_description
    aspect = laptops_aspects_list

    input_text = f"The sentence is: \"{sentence}\" and the Aspect is: \"{aspect}\""

    inputs = laptops_sentiment_tokenizer(
    [
      alpaca_prompt.format(
          laptops_sentiment_instruction_prompt,
          input_text, # input
          "", # output - leave this blank for generation!
      )
    ], return_tensors = "pt").to("cuda")

    outputs = laptops_sentiment_model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    output = laptops_sentiment_tokenizer.batch_decode(outputs)

    # output = pre_process_output(output)
    aspect_sentiment_laptops = get_aspect_sentiment_restaurants(output, aspect)

    return aspect_sentiment_laptops

# Now we defined the functon for our APIs

In [ ]:
def generate_aspects(domain,model, user_description):

    if domain == "restaurants":
        output = get_aspect_restaurants(user_description)

        list_aspects = get_aspect_restaurants_list(output)
    else:
        output = get_aspect_laptops(user_description)

        list_aspects = get_aspect_laptops_list(output)

    return list_aspects

In [ ]:
# get_sentiment_restaurants(user_description, restaurants_aspects_list)

def get_sentiments(domain, model, user_description, list_aspects):

    if domain == "restaurants":
        aspect_sentiment_list = get_sentiment_restaurants(user_description, list_aspects)
    else:
        aspect_sentiment_list = get_sentiment_laptops(user_description, list_aspects)

    return aspect_sentiment_list

In [ ]:
@app.route('/get_aspects', methods=['POST'])
def get_aspects_api():

    data = request.get_json()

    domain = data.get('domain', None)
    model = data.get('model', None)
    user_description = data.get('user_description', None)

    list_aspects = generate_aspects(domain,model, user_description)

    list_aspects_sentiments = get_sentiments(domain,model, user_description, list_aspects)

    return_message = {
        "list_aspects_sentiments": list_aspects_sentiments
    }

    return jsonify(return_message)

In [ ]:
@app.route('/get_aspects2', methods=['POST'])
def get_aspects_api2():

    data = request.get_json()

    domain = data.get('domain', None)
    model = data.get('model', None)
    user_description = data.get('user_description', None)

    print("***********************************************************")
    print(f"Domain: {domain}, model: {model}, user description: {user_description}")
    print("***********************************************************")


    list_aspects = generate_aspects(domain,model, user_description)

    list_aspects_sentiments = get_sentiments(domain,model, user_description, list_aspects)

    return_message = {
        "list_aspects_sentiments": list_aspects_sentiments
    }

    return jsonify(return_message)

In [ ]:
print("Add the following api url to Aspectify, so you can accss the backend")
print("Backend API URL: ",public_url)

In [ ]:
if __name__ == '__main__':
    app.run(port=PORT_NUMBER)